In [50]:
import pandas as pd
from pathlib import Path
import zipfile
import os
import sqlite3
from datetime import datetime
import glob
from pprint import pprint
import praw
import pprint
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import string
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

import contractions
import re
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from collections import Counter
import gensim
from gensim import corpora
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
import gensim.downloader as api
from gensim.models.ldamodel import LdaModel

from dateutil.parser import parse
from datetime import datetime
from pandas import json_normalize

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
from textblob import TextBlob

In [51]:
import pandas as pd
import json
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import spacy
from nltk.stem import WordNetLemmatizer
import os
import http.client
import urllib.parse
import json
import pandas as pd
from datetime import datetime
import re
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
os.getcwd() 

'c:\\Users\\PRAGYA\\MSC Project\\Streamlit\\stock-news-prediction'

In [74]:
def fetch_news():
    # Get today's date in YYYY-MM-DD format
    today = datetime.today().strftime('%Y-%m-%d')
    file_name = f'news_data_{today}.json'

    # Define the parent directory and file path
    # parent_dir = os.path.dirname(os.path.abspath(__file__))

    file_path = os.path.join(os.getcwd() , "data", file_name)

    # Check if the file for today's date exists
    if os.path.exists(file_path):
        # Load the JSON file
        with open(file_path, 'r', encoding='utf-8') as json_file:
            news_data = json.load(json_file)
            print(news_data)
        print(f"Loaded data from {file_name}")
    else:
        # Fetch news data from the API
        conn = http.client.HTTPConnection('api.mediastack.com')
        params = urllib.parse.urlencode({
            # 'access_key': '65c5e9342fc45dc08165bf28b32d8ccf',
            'categories': 'business',
            'sort': 'published_desc',
            "language": "en",
            "country": "gb"
        })
        conn.request('GET', '/v1/news?{}'.format(params))
        res = conn.getresponse()
        data = res.read()
        news_data = json.loads(data.decode('utf-8'))

        # Save the fetched data to a JSON file
        os.makedirs(os.path.join(parent_dir, "data"), exist_ok=True)
        with open(file_path, 'w', encoding='utf-8') as json_file:
            json.dump(news_data, json_file, ensure_ascii=False, indent=4)
        print(f"Fetched and saved data to {file_name}")

    # Convert the JSON data to a DataFrame
    news_df = pd.json_normalize(news_data['data'])
    
    # Optional: Convert 'published_at' to date if needed
    news_df['date'] = pd.to_datetime(news_df['published_at']).dt.date
    
    return news_df

In [75]:
news_df = fetch_news()
news_df

{'pagination': {'limit': 25, 'offset': 0, 'count': 25, 'total': 10000}, 'data': [{'author': 'Bloomberg News', 'title': 'Asia Eyes Cautious Open After China Factory Data: Markets Wrap', 'description': 'Asian stocks are set for a cautious start into a historically volatile month for markets as signs mount that China’s efforts to support its ailing economy are yet to take hold.', 'url': 'https://financialpost.com/pmn/business-pmn/asia-eyes-cautious-open-after-china-factory-data-markets-wrap', 'source': 'Financial Post | Canada Business News', 'image': None, 'category': 'business', 'language': 'en', 'country': 'us', 'published_at': '2024-09-01T22:55:17+00:00'}, {'author': 'Chris Roush', 'title': 'Energy reporter Zullo moving to national editing position', 'description': 'States Newsroom national energy reporter Robert Zullo\xa0is moving to a national editing position for the news organization. Zullo has been a national energy reporter for States Newsroom based in southern Illinois focusing

,author,title,description,url,source,image,category,language,country,published_at,date
0,Bloomberg News,Asia Eyes Cautious Open After China Factory Da...,Asian stocks are set for a cautious start into...,https://financialpost.com/pmn/business-pmn/asi...,Financial Post | Canada Business News,None,business,en,us,2024-09-01T22:55:17+00:00,2024-09-01
1,Chris Roush,Energy reporter Zullo moving to national editi...,States Newsroom national energy reporter Rober...,https://talkingbiznews.com/media-news/energy-r...,Talking Biz News,None,business,en,us,2024-09-01T21:52:13+00:00,2024-09-01
2,Reuters,Exclusive-Intel CEO to pitch board on plans to...,Exclusive-Intel CEO to pitch board on plans to...,https://www.investing.com/news/stock-market-ne...,Investing.com | Stock Market Quotes &amp; Fina...,https://i-invdn-com.investing.com/trkd-images/...,business,en,us,2024-09-01T21:45:54+00:00,2024-09-01
3,Giovanna Sutto,Musk faz agradecimento a Arthur Lira após crít...,Musk fez um post no X ressaltando que Lira apo...,https://www.infomoney.com.br/politica/musk-faz...,InfoMoney,None,business,pt,br,2024-09-01T21:22:38+00:00,2024-09-01
4,None,"Superstars Clooney, Pitt Spar And Tease In Lon...",George Clooney and Brad Pitt brought a jolt of...,https://www.ibtimes.com.au/superstars-clooney-...,International Business Times Au,None,business,en,au,2024-09-01T20:57:06+00:00,2024-09-01
5,None,UK government to probe ‘dynamic’ pricing behin...,Cost jumped by hundreds of pounds during onlin...,https://www.ft.com/content/0c728ded-176a-43b1-...,Financial Times,https://www.ft.com/__origami/service/image/v2/...,business,en,us,2024-09-01T20:56:36+00:00,2024-09-01
6,Amanda Holpuch,Man Sentenced for Harassing New Hampshire Publ...,Tucker Cockerline was part of a group that pro...,https://www.nytimes.com/2024/09/01/business/me...,The New York Times,https://static01.nyt.com/images/2024/09/01/us/...,business,en,us,2024-09-01T20:50:00+00:00,2024-09-01
7,None,A Popular Lopez Obrador Presents Final Report ...,Mexico's President Andres Manuel Lopez Obrador...,https://www.ibtimes.com.au/popular-lopez-obrad...,International Business Times Au,None,business,en,au,2024-09-01T20:33:06+00:00,2024-09-01
8,None,Is September the most bearish month? Yes for t...,Is September the most bearish month? Yes for t...,https://seekingalpha.com/news/4145246-is-septe...,Seeking Alpha,None,business,en,us,2024-09-01T20:22:00+00:00,2024-09-01
9,None,Andhra Pradesh rains: Heavy rainfall continues...,Heavy rainfall triggers floods and landslides ...,https://www.thehindubusinessline.com/multimedi...,Business Line,https://bl-i.thgim.com/public/news/vv8sca/arti...,business,en,us,2024-09-01T20:11:43+00:00,2024-09-01


In [76]:
news_df = news_df[news_df['language'] =='en']
news_df

,author,title,description,url,source,image,category,language,country,published_at,date
0,Bloomberg News,Asia Eyes Cautious Open After China Factory Da...,Asian stocks are set for a cautious start into...,https://financialpost.com/pmn/business-pmn/asi...,Financial Post | Canada Business News,None,business,en,us,2024-09-01T22:55:17+00:00,2024-09-01
1,Chris Roush,Energy reporter Zullo moving to national editi...,States Newsroom national energy reporter Rober...,https://talkingbiznews.com/media-news/energy-r...,Talking Biz News,None,business,en,us,2024-09-01T21:52:13+00:00,2024-09-01
2,Reuters,Exclusive-Intel CEO to pitch board on plans to...,Exclusive-Intel CEO to pitch board on plans to...,https://www.investing.com/news/stock-market-ne...,Investing.com | Stock Market Quotes &amp; Fina...,https://i-invdn-com.investing.com/trkd-images/...,business,en,us,2024-09-01T21:45:54+00:00,2024-09-01
4,None,"Superstars Clooney, Pitt Spar And Tease In Lon...",George Clooney and Brad Pitt brought a jolt of...,https://www.ibtimes.com.au/superstars-clooney-...,International Business Times Au,None,business,en,au,2024-09-01T20:57:06+00:00,2024-09-01
5,None,UK government to probe ‘dynamic’ pricing behin...,Cost jumped by hundreds of pounds during onlin...,https://www.ft.com/content/0c728ded-176a-43b1-...,Financial Times,https://www.ft.com/__origami/service/image/v2/...,business,en,us,2024-09-01T20:56:36+00:00,2024-09-01
6,Amanda Holpuch,Man Sentenced for Harassing New Hampshire Publ...,Tucker Cockerline was part of a group that pro...,https://www.nytimes.com/2024/09/01/business/me...,The New York Times,https://static01.nyt.com/images/2024/09/01/us/...,business,en,us,2024-09-01T20:50:00+00:00,2024-09-01
7,None,A Popular Lopez Obrador Presents Final Report ...,Mexico's President Andres Manuel Lopez Obrador...,https://www.ibtimes.com.au/popular-lopez-obrad...,International Business Times Au,None,business,en,au,2024-09-01T20:33:06+00:00,2024-09-01
8,None,Is September the most bearish month? Yes for t...,Is September the most bearish month? Yes for t...,https://seekingalpha.com/news/4145246-is-septe...,Seeking Alpha,None,business,en,us,2024-09-01T20:22:00+00:00,2024-09-01
9,None,Andhra Pradesh rains: Heavy rainfall continues...,Heavy rainfall triggers floods and landslides ...,https://www.thehindubusinessline.com/multimedi...,Business Line,https://bl-i.thgim.com/public/news/vv8sca/arti...,business,en,us,2024-09-01T20:11:43+00:00,2024-09-01
10,Reuters,Analysis-Far-right state win shakes Germany's ...,Analysis-Far-right state win shakes Germany's ...,https://www.investing.com/news/world-news/anal...,Investing.com | Stock Market Quotes &amp; Fina...,https://i-invdn-com.investing.com/trkd-images/...,business,en,us,2024-09-01T20:10:42+00:00,2024-09-01


In [11]:
news_df.rename(columns = {'title':'headline','description':'short_description'},inplace=True)
news_df = news_df[['date','headline','category','short_description','source']]
news_df

,date,headline,category,short_description,source
1,2024-08-31,Harris: Trump 'Disrespected Sacred Ground' At ...,business,Democratic presidential candidate Kamala Harri...,International Business Times Au
4,2024-08-31,Telegraph taps Kelly as associate money editor,business,The Telegraph in London has hired Helena Kelly...,Talking Biz News
6,2024-08-31,Harris calls Trump cemetery visit disrespectfu...,business,Harris calls Trump cemetery visit disrespectfu...,Investing.com | Stock Market Quotes &amp; Fina...
8,2024-08-31,UK government introduces AI training courses f...,business,The UK Government has launched nine AI trainin...,Business Matters | The Uk's Leading Sme Busine...
11,2024-08-31,"Executive reshuffles: CMG, TSN and CODI Compan...",business,"Executive reshuffles: CMG, TSN and CODI Compan...",Seeking Alpha
13,2024-08-31,US Jobs Data Will Help Fed Gauge Extent of Mod...,business,"Upcoming readouts on the US labor market, incl...",Financial Post | Canada Business News
14,2024-08-31,"UK, Ireland Oasis Reunion Concerts Sell Out Af...",business,Oasis's reunion concerts in the UK and Ireland...,International Business Times Au
18,2024-08-31,5 people who inspired you during the early sta...,business,My business and work combine a variety of disc...,Business Matters | The Uk's Leading Sme Busine...
21,2024-08-31,"Notable analyst calls this week: Nvidia, Barri...",business,"Notable analyst calls this week: Nvidia, Barri...",Seeking Alpha
24,2024-08-31,Amazon to hire founders of robot software make...,business,Amazon to hire founders of robot software make...,Seeking Alpha


In [12]:
stop_words = set(stopwords.words('english'))
nlp = spacy.load("en_core_web_sm")

In [13]:
add_words = {'news','read','story','world','newsbrief','huffpost','short',
            'wrapup'}
stop_words = stop_words.union(add_words)
phrases_to_remove = [
    'new york city', 
    'new york times', 
    'rise morning',
    'huffpost rise morning newsbrief', 
    'morning newsbrief short wrapup',
    'welcome to the huffPost rise Morning Newsbrief',
    'a short wrap-up of the news to help you start your day',
    'welcome huffpost rise morning newsbrief short wrapup help start day',
    'HuffPost Rise What You Need To Know On',
]

news_source_words = news_df['source'].unique()
phrases_to_remove.extend(news_source_words)
phrases_to_remove

['new york city',
 'new york times',
 'rise morning',
 'huffpost rise morning newsbrief',
 'morning newsbrief short wrapup',
 'welcome to the huffPost rise Morning Newsbrief',
 'a short wrap-up of the news to help you start your day',
 'welcome huffpost rise morning newsbrief short wrapup help start day',
 'HuffPost Rise What You Need To Know On',
 'International Business Times Au',
 'Talking Biz News',
 'Investing.com | Stock Market Quotes &amp; Financial News',
 "Business Matters | The Uk's Leading Sme Business Magazine",
 'Seeking Alpha',
 'Financial Post | Canada Business News']

In [18]:
# Function to clean text
def clean_text(text):
    if not isinstance(text, str):
        return ""

    # Contractions expansion
    try:
        text = contractions.fix(text)
    except:
        print(text)
    
    # 1. Convert text to lowercase
    text = text.lower()

    # 2. Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # 3. Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # 4. Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # 5. Remove numbers
    text = re.sub(r'\d+', '', text)
    # 5.0 Remove non-alphanumeric characters (except spaces)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub('[\?|$|.|!-/,]',"", text)
    
    # 5.1. Remove specific phrases
    for phrase in phrases_to_remove:
        text = re.sub(re.escape(phrase.lower()), '', text)

    # 6. Tokenize text
    tokens = word_tokenize(text)

    # 7. Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]

    # 8. Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # 9. Remove special characters 
    tokens = [re.sub(r'\W+', '', word) for word in tokens]
    
    # Remove single alphabet characters
    tokens = [word for word in tokens if len(word) > 1 and word.isalpha()]

    # 10. Remove extra whitespace and join tokens back into a single string
    cleaned_text = ' '.join(tokens).strip()

    return cleaned_text

<>:28: SyntaxWarning: invalid escape sequence '\?'
<>:28: SyntaxWarning: invalid escape sequence '\?'
C:\Users\PRAGYA\AppData\Local\Temp\ipykernel_13936\2879768049.py:28: SyntaxWarning: invalid escape sequence '\?'
  text = re.sub('[\?|$|.|!-/,]',"", text)


In [19]:
news_df['combined'] = news_df['headline'] + news_df['short_description']
news_df['combined'] = news_df['combined'].fillna('').astype(str)
combined_df = news_df.groupby('date').agg({
    'combined': ' '.join
}).reset_index()
combined_df

C:\Users\PRAGYA\AppData\Local\Temp\ipykernel_13936\1027064068.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['combined'] = news_df['headline'] + news_df['short_description']
C:\Users\PRAGYA\AppData\Local\Temp\ipykernel_13936\1027064068.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  news_df['combined'] = news_df['combined'].fillna('').astype(str)


,date,combined
0,2024-08-31,Harris: Trump 'Disrespected Sacred Ground' At ...


In [20]:
combined_df['clean_title'] = combined_df['combined'].apply(clean_text)

Harris: Trump 'Disrespected Sacred Ground' At US Military CemeteryDemocratic presidential candidate Kamala Harris said Saturday that her Republican rival Donald Trump "disrespected sacred ground" at a US military cemetery during a contentious pre-election visit. Telegraph taps Kelly as associate money editorThe Telegraph in London has hired Helena Kelly to be associate money editor. Kelly previously was U.S. assistant consumer editor for the Daily Mail in New York, covering stories related to fraud, banking, property, 401(K)s and general consumer issues. She started as a trainee on the Daily Mail&#8217;s print graduate scheme in September 2019 and [&#8230;] Harris calls Trump cemetery visit disrespectful, political stuntHarris calls Trump cemetery visit disrespectful, political stunt UK government introduces AI training courses for civil servants to boost public sector efficiencyThe UK Government has launched nine AI training courses for Civil Servants, focusing on AI ethics, generativ

In [21]:
comment_combined =''
for each in combined_df['clean_title']:
    comment_combined = comment_combined + " " + each

In [22]:
startofdoc = 0
endofdoc = 1000000

df_ner = pd.DataFrame(columns=['Text','Label'])

while startofdoc < len(comment_combined) :
    print(startofdoc, endofdoc)
    doc = nlp(comment_combined[startofdoc:endofdoc])

    for ent in doc.ents:
        df_ner.loc[len(df_ner)]  = [ent.text, ent.label_]
    startofdoc = startofdoc + 1000000 + 1
    endofdoc = endofdoc + 1000000 - 2

0 1000000


In [28]:
for name_val in df_ner['Label'].unique():
    try:
        xdf = pd.DataFrame(df_ner[df_ner['Label'] ==name_val]['Text'].value_counts()[:10]).reset_index()
        xdf = xdf.rename(columns = {'index':name_val})
        xdf = xdf.rename(columns = {'Text':'frequency'})
        plt.barh(xdf[name_val], xdf.frequency)

        # setting label of y-axis
        plt.ylabel( name_val + " Entity Analysis")

        # setting label of x-axis
        plt.xlabel("Frequency")
        plt.title(name_val + " Entity Analysis")
        plt.show()
    except: pass

In [24]:
df_topic = combined_df.copy()

In [25]:
# Preprocessing Function
def preprocess_news(news):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    
    # Tokenization
    tokens = word_tokenize(news.lower())
    
    # Removing stopwords and lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    
    return tokens

# Apply preprocessing
df_topic['tokens'] = df_topic['clean_title'].apply(preprocess_news)
df_topic

,date,combined,clean_title,tokens
0,2024-08-31,Harris: Trump 'Disrespected Sacred Ground' At ...,harris trump disrespected sacred ground milita...,"[harris, trump, disrespected, sacred, ground, ..."


In [33]:
import nltk
nltk.download('stopwords')
import gensim
from gensim import corpora

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PRAGYA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
# Create a dictionary representation of the documents.
dictionary = corpora.Dictionary(df_topic['tokens'])

# Create a corpus: List of lists of (token_id, token_count)
corpus = [dictionary.doc2bow(text) for text in df_topic['tokens']]

In [35]:
optimal_num_topics = 16

In [37]:
from gensim.models.ldamodel import LdaModel

In [38]:
# Train the LDA model
lda_model = LdaModel(corpus=corpus, 
                     id2word=dictionary, 
                     num_topics=optimal_num_topics, 
                     random_state=42, 
                     passes=10,
                    update_every=1,
                    alpha='auto',
                    per_word_topics=True
                    )

# Display topics and their associated words
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(f"Topic {topic[0]}: {topic[1]}")

Topic 0: 0.006*"uk" + 0.006*"ai" + 0.006*"trump" + 0.006*"visit" + 0.006*"boost"
Topic 1: 0.006*"trump" + 0.006*"uk" + 0.006*"people" + 0.006*"general" + 0.006*"ai"
Topic 2: 0.006*"call" + 0.006*"uk" + 0.006*"ai" + 0.006*"people" + 0.006*"boost"
Topic 3: 0.006*"uk" + 0.006*"people" + 0.006*"trump" + 0.006*"boost" + 0.006*"kelly"
Topic 4: 0.006*"uk" + 0.006*"trump" + 0.006*"ai" + 0.006*"people" + 0.006*"call"
Topic 5: 0.006*"uk" + 0.006*"ai" + 0.006*"trump" + 0.006*"call" + 0.006*"business"
Topic 6: 0.018*"uk" + 0.014*"trump" + 0.014*"call" + 0.014*"ai" + 0.014*"people"
Topic 7: 0.005*"uk" + 0.005*"visit" + 0.005*"people" + 0.005*"trump" + 0.005*"civil"
Topic 8: 0.006*"training" + 0.006*"uk" + 0.006*"people" + 0.006*"call" + 0.006*"kelly"
Topic 9: 0.006*"trump" + 0.006*"ai" + 0.006*"uk" + 0.006*"people" + 0.006*"sector"
Topic 10: 0.006*"people" + 0.006*"uk" + 0.006*"call" + 0.006*"ai" + 0.006*"training"
Topic 11: 0.006*"people" + 0.006*"trump" + 0.006*"ai" + 0.006*"government" + 0.006*"

In [39]:
# Get the topics from the best LDA model
topics = lda_model.print_topics(-1)

# Function to clean up the topics
def clean_topic(topic):
    # Remove the numeric weights (e.g., '0.007*') using a regular expression
    cleaned_topic = ''.join([word.split("*")[1] for word in topic.split(" + ")])
    cleaned_topic = cleaned_topic.replace('"',' ')
    return cleaned_topic.strip(' ')  # Remove any extra quotation marks

for topic in topics:
    print(f"Topic {topic[0]}: {clean_topic(topic[1])}")
    

Topic 0: uk  ai  trump  visit  boost  week  civil  public  servant  call
Topic 1: trump  uk  people  general  ai  harris  visit  call  servant  business
Topic 2: call  uk  ai  people  boost  trump  week  course  sector  public
Topic 3: uk  people  trump  boost  kelly  inspired  sector  general  ai  visit
Topic 4: uk  trump  ai  people  call  inspired  visit  harris  cemetery  week
Topic 5: uk  ai  trump  call  business  people  cemetery  civil  government  kelly
Topic 6: uk  trump  call  ai  people  sector  course  inspired  general  civil
Topic 7: uk  visit  people  trump  civil  ai  week  cemetery  servant  call
Topic 8: training  uk  people  call  kelly  ai  visit  public  cemetery  boost
Topic 9: trump  ai  uk  people  sector  visit  call  harris  week  kelly
Topic 10: people  uk  call  ai  training  inspired  course  government  kelly  business
Topic 11: people  trump  ai  government  week  uk  civil  call  general  inspired
Topic 12: presidential  print  organising  payroll  peop

In [40]:
# Assign topic distribution to each day
def get_topic_distribution(text):
    bow = dictionary.doc2bow(text)
    return lda_model.get_document_topics(bow)

df_topic['topic_distribution'] = df_topic['tokens'].apply(get_topic_distribution)

df_topic

,date,combined,clean_title,tokens,topic_distribution
0,2024-08-31,Harris: Trump 'Disrespected Sacred Ground' At ...,harris trump disrespected sacred ground milita...,"[harris, trump, disrespected, sacred, ground, ...","[(6, 0.99840814)]"


In [41]:
from collections import defaultdict

def aggregate_topics_by_date(group):
    topic_sums = defaultdict(float)
    for dist in group['topic_distribution']:
        for topic_id, prob in dist:
            topic_sums[topic_id] += prob
    total = sum(topic_sums.values())
    return {f"topic_{k}": v/total for k, v in topic_sums.items()}

# Group by date and aggregate topics
topic_features = df_topic.groupby('date').apply(aggregate_topics_by_date).apply(pd.Series).reset_index()

print(topic_features)


         date  topic_6
0  2024-08-31      1.0


C:\Users\PRAGYA\AppData\Local\Temp\ipykernel_13936\2690116892.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  topic_features = df_topic.groupby('date').apply(aggregate_topics_by_date).apply(pd.Series).reset_index()


In [42]:
df_ner_columns = [ 'NORP', 'PERSON', 'ORG','CARDINAL','GPE',
        'ORDINAL','TIME', 'LOC', 'EVENT', 'LAW',  'FAC',
                  'MONEY','PRODUCT', 'QUANTITY', 'PERCENT']
top_ner = 5

In [43]:

df_ner = df_ner[df_ner['Label'].isin(df_ner_columns)]
df_ner = df_ner.rename(columns= {'Text':'entity','Label':'label'})
df_ner

,entity,label
0,kamala harris,PERSON
2,republican,NORP
3,donald trump,PERSON
4,london,GPE
5,kelly,PERSON
7,new york,GPE
8,trainee daily,ORG
10,nine,CARDINAL
13,uk ireland,GPE
15,british,NORP


In [44]:
# Group by 'label' and 'entity' to count occurrences
entity_counts = df_ner.groupby(['label', 'entity']).size().reset_index(name='count')

# For each label, get the top 3 entities based on frequency
top_entities = entity_counts.groupby('label').apply(lambda x: x.nlargest(top_ner, 'count')).reset_index(drop=True)

# Convert to a dictionary for easy lookup
top_entities_dict = top_entities.groupby('label')['entity'].apply(list).to_dict()

# Display the top entities for each label
print(top_entities_dict)

{'CARDINAL': ['nine'], 'GPE': ['london', 'new york', 'uk ireland'], 'NORP': ['british', 'republican'], 'ORG': ['week nvidia barrick gold dollar', 'trainee daily'], 'PERSON': ['donald trump', 'kamala harris', 'kelly']}


C:\Users\PRAGYA\AppData\Local\Temp\ipykernel_13936\1724077355.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_entities = entity_counts.groupby('label').apply(lambda x: x.nlargest(top_ner, 'count')).reset_index(drop=True)


In [45]:
df_ner_feat = combined_df.copy()
df_ner_feat

,date,combined,clean_title
0,2024-08-31,Harris: Trump 'Disrespected Sacred Ground' At ...,harris trump disrespected sacred ground milita...


In [46]:
# Initialize columns for the top NER entities
for label, entities in top_entities_dict.items():
    for entity in entities:
        df_ner_feat[f'{label}_{entity}'] = df_ner_feat['clean_title'].apply(lambda x: 1 if entity in x else 0)

# Display the DataFrame with new features
df_ner_feat.head()

,date,combined,clean_title,CARDINAL_nine,GPE_london,GPE_new york,GPE_uk ireland,NORP_british,NORP_republican,ORG_week nvidia barrick gold dollar,ORG_trainee daily,PERSON_donald trump,PERSON_kamala harris,PERSON_kelly
0,2024-08-31,Harris: Trump 'Disrespected Sacred Ground' At ...,harris trump disrespected sacred ground milita...,1,1,1,1,1,1,1,1,1,1,1


In [47]:
df_clust = combined_df.copy()
df_clust

,date,combined,clean_title
0,2024-08-31,Harris: Trump 'Disrespected Sacred Ground' At ...,harris trump disrespected sacred ground milita...


In [48]:

# Function to extract entities and key phrases
def extract_phrases(text):
    doc = nlp(text)
    # Extract named entities and noun chunks (key phrases)
    entities = [ent.text for ent in doc.ents]
    noun_chunks = [chunk.text for chunk in doc.noun_chunks]
    return entities + noun_chunks

# Apply the function to the news articles
df_clust['phrases'] = df_clust['clean_title'].apply(extract_phrases)
df_clust

,date,combined,clean_title,phrases
0,2024-08-31,Harris: Trump 'Disrespected Sacred Ground' At ...,harris trump disrespected sacred ground milita...,"[harris trump, kamala harris, saturday, republ..."


In [52]:

# Flatten the list of phrases for vectorization
df_clust['phrases_flat'] = df_clust['phrases'].apply(lambda x: ' '.join(x))

# Vectorize the phrases
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_clust['phrases_flat'])

In [53]:
best_eps = 200
best_min_samples = 2

In [54]:
from sklearn.cluster import DBSCAN

In [55]:
# Apply DBSCAN clustering
dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples)
df_clust['event_cluster'] = dbscan.fit_predict(X.toarray())

# Label events as significant if they belong to a cluster
df_clust['is_event'] = df_clust['event_cluster'].apply(lambda x: 1 if x != -1 else 0)
df_clust

,date,combined,clean_title,phrases,phrases_flat,event_cluster,is_event
0,2024-08-31,Harris: Trump 'Disrespected Sacred Ground' At ...,harris trump disrespected sacred ground milita...,"[harris trump, kamala harris, saturday, republ...",harris trump kamala harris saturday republican...,-1,0


In [57]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
def score_vader(text):
    return sia.polarity_scores(text)["compound"]

In [58]:
combined_df['vader_sentiment'] = combined_df['clean_title'].apply(score_vader)
combined_df.head()

,date,combined,clean_title,vader_sentiment
0,2024-08-31,Harris: Trump 'Disrespected Sacred Ground' At ...,harris trump disrespected sacred ground milita...,0.946


In [61]:
!pip install textblob


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [64]:

from textblob import TextBlob
def score_textblob(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity  # You can also include subjectivity if needed

combined_df['textblob_sentiment'] = combined_df['clean_title'].apply(score_textblob)


In [67]:
df_clean = combined_df.copy()

In [68]:
df_clean = df_clean[['date','vader_sentiment','textblob_sentiment']]
df_clean = df_clean.merge(df_clust[['date','is_event']], on ='date')
df_clean = df_clean.merge(topic_features, on ='date')
df_clean = df_clean.merge(df_ner_feat, on ='date')
df_clean.drop(columns=['combined','clean_title'],inplace=True)
df_clean.fillna(0,inplace=True)
df_clean

,date,vader_sentiment,textblob_sentiment,is_event,topic_6,CARDINAL_nine,GPE_london,GPE_new york,GPE_uk ireland,NORP_british,NORP_republican,ORG_week nvidia barrick gold dollar,ORG_trainee daily,PERSON_donald trump,PERSON_kamala harris,PERSON_kelly
0,2024-08-31,0.946,0.004721,0,1.0,1,1,1,1,1,1,1,1,1,1,1


In [65]:
combined_feature = ['Price_lag_1d',
 'Price_lag_4d',
 'Price_RollingAvg_5d',
 'vader_sentiment',
 'TIME_hour',
 'textblob_sentiment',
 'topic_15',
 'GPE_russia',
 'is_event',
 'CARDINAL_two',
 'High_Low_Ratio',
 'Price_RollingAvg_14d',
 'Vol.',
 'Price_lag_12d',
 'Price_RollingAvg_7d',
 'Price_lag_2d',
 'PERSON_putin',
 'Price_RollingAvg_3d',
 'Price_Per_Volume',
 'TIME_night',
 'PERSON_bernie',
 'Price_Volume',
 'Price_Range',
 'CARDINAL_million',
 'Price_lag_10d',
 'Price_lag_8d',
 'Low',
 'CARDINAL_three']

In [100]:
import requests
from bs4 import BeautifulSoup


In [101]:
def fetch_ftse_100_price():
    # URL of the website you want to scrape
    url = "https://finance.yahoo.com/quote/%5EFTSE/"

    # Send a GET request to the website
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Locate the element that contains the FTSE 100 price
        # The exact selector will depend on the website's structure
        price_tag = soup.find('div', class_='price')  # This is an example, you need to inspect the page to find the correct class or tag

        if price_tag:
            price = price_tag.text.strip()
            # st.write(f"FTSE 100 Index Price: {price}")
            return price
        else:
            print("Could not find the FTSE 100 price on the page.")
            return None
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

# Example usage
price  = fetch_ftse_100_price()
price

'8,376.63 -3.01 (-0.04%) At close: August 30 at 4:35 PM GMT+1'

In [98]:
for each in df_clean.columns:
    if each in combined_feature:
        print(each)


vader_sentiment
textblob_sentiment
is_event


In [82]:
import requests
def fetch_stock_data(symbol):
    # Define the filename based on the symbol
    today = datetime.today().strftime('%Y-%m-%d')
    file_name = f'stocks_{symbol}_{today}.json'
    
    # Define the directory where the file will be saved
    # parent_dir = os.path.dirname(os.path.abspath(__file__))
    parent_dir = os.getcwd()
    data_dir = os.path.join(parent_dir, "data")
    file_path = os.path.join(data_dir, file_name)

    # Check if the file already exists
    if os.path.exists(file_path):
        # Read the data from the existing file
        with open(file_path, 'r', encoding='utf-8') as json_file:
            stock_data = json.load(json_file)
        print(f"Loaded data from {file_name}")
    else:
        # Fetch data from the API
        url = "https://api.marketstack.com/v1/eod"
        params = {
            "access_key": "d8be15dd3062413737a6e838bab6801a",
            "symbols": symbol
        }
        response = requests.get(url, params=params)
        
        # Check if the request was successful
        if response.status_code == 200:
            stock_data = response.json()

            # Ensure the directory exists
            os.makedirs(data_dir, exist_ok=True)
            
            # Save the fetched data to a file
            with open(file_path, 'w', encoding='utf-8') as json_file:
                json.dump(stock_data, json_file, ensure_ascii=False, indent=4)
            print(f"Fetched and saved data to {file_name}")
        else:
            print(f"Failed to fetch data: {response.status_code}")
            return None
    
    # Convert stock data to DataFrame
    df = pd.DataFrame(stock_data['data'])  # Assuming the stock data is under the 'data' key
    
    return df

In [88]:
# Example usage:
symbol = "UKX"
df_stock_data = fetch_stock_data(symbol)

# Optionally, display the DataFrame
print(df_stock_data)

Failed to fetch data: 422
None


In [95]:
url = "https://finance.yahoo.com/quote/%5EFTSE/"

# Send a GET request to the website
response = requests.get(url)
response

<Response [200]>

In [105]:
import requests
from bs4 import BeautifulSoup

def fetch_ftse_100_price():
    # URL of the website you want to scrape
    url = "https://finance.yahoo.com/quote/%5EFTSE/"

    # Send a GET request to the website
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')

        # Locate the element that contains the FTSE 100 price
        # The exact selector will depend on the website's structure
        price_tag = soup.find('div', class_='price')  # This is an example, you need to inspect the page to find the correct class or tag

        if price_tag:
            price = price_tag.text.strip()
            # st.write(f"FTSE 100 Index Price: {price}")
            return price
        else:
            print("Could not find the FTSE 100 price on the page.")
            return None
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

price_string = fetch_ftse_100_price()

# Print the result
print(price_string)

8,376.63 -3.01 (-0.04%) At close: August 30 at 4:35 PM GMT+1


In [107]:
price_string = price_string.split(' ')[0]

# Remove any commas and convert to float
price_value = float(price_string.replace(',', ''))

# Print the result
print(price_value)

8376.63


In [155]:
df=pd.read_csv(r'C:\Users\PRAGYA\MSC Project\Streamlit\stock-news-prediction\data\historical_data.csv')
# df.drop(columns=['Unnamed: 0'],inplace=True)
df['date'] = pd.to_datetime(df['time_index'])

# # Step 2: Now, you can format the 'date' column to 'dd/mm/yyyy'
df['date'] = df['date'].dt.strftime('%d/%m/%Y')
df.sort_values(by='date')
df

,time_index,topic_15_lag_1d,High_Low_Ratio_RollingAvg_14d,vader_sentiment_RollingAvg_3d,Vol._RollingAvg_14d,NORP_russian_RollingAvg_5d,Low_RollingAvg_14d,Vol._RollingAvg_3d,Vol._lag_10d,Low,...,High_Low_Ratio_RollingAvg_3d,textblob_sentiment_RollingAvg_7d,Low_lag_1d,Price_Range,Low_RollingAvg_3d,LAW_west ham_RollingAvg_5d,Low_RollingAvg_7d,ORDINAL_first_RollingAvg_14d,topic_15_RollingAvg_14d,date
0,2012-01-28,0.045760,1.011591,-0.044376,793.514555,0.143903,6829.312064,793.514555,793.514555,5728.96,...,1.011591,0.065850,6829.312064,66.24,6829.312064,0.004347,6829.312064,0.452986,0.045760,28/01/2012
1,2012-01-29,0.129553,1.011591,-0.044376,793.514555,0.143903,6829.312064,793.514555,793.514555,5728.96,...,1.011591,0.065850,5728.960000,66.24,6829.312064,0.004347,6829.312064,0.452986,0.045760,29/01/2012
2,2012-01-30,0.029583,1.011591,0.990100,793.514555,0.143903,6829.312064,786.066667,793.514555,5651.56,...,1.012538,0.065850,5728.960000,81.89,5703.160000,0.004347,6829.312064,0.452986,0.045760,30/01/2012
3,2012-01-31,0.108482,1.011591,0.984100,793.514555,0.143903,6829.312064,862.716667,793.514555,5671.09,...,1.012165,0.065850,5651.560000,59.23,5683.870000,0.004347,6829.312064,0.452986,0.045760,31/01/2012
4,2012-02-01,0.058440,1.011591,0.984967,793.514555,0.200000,6829.312064,927.783333,793.514555,5680.67,...,1.014769,0.065850,5671.090000,110.05,5667.773333,0.000000,6829.312064,0.452986,0.045760,01/02/2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4366,2024-04-26,0.456016,1.010221,-0.269667,959.960000,0.000000,7928.100714,1053.866667,838.700000,8078.86,...,1.008446,-0.007187,8030.050000,67.93,8046.900000,0.000000,7953.940000,0.214286,0.434092,26/04/2024
4367,2024-05-04,0.550672,1.010494,-0.278033,937.587143,0.000000,7945.300000,838.320000,710.690000,8172.15,...,1.009062,0.012015,8078.860000,76.58,8093.686667,0.000000,8005.721429,0.214286,0.448035,04/05/2024
4368,2024-06-04,0.842991,1.010648,-0.258733,951.402857,0.000000,7964.507143,831.653333,973.750000,8200.26,...,1.008492,-0.030842,8172.150000,63.12,8150.423333,0.000000,8061.518571,0.214286,0.431611,04/06/2024
4369,2024-07-04,0.524685,1.010931,-0.428933,914.877143,0.200000,7981.192857,750.666667,1150.000000,8170.97,...,1.009238,-0.039908,8200.260000,86.98,8181.126667,0.000000,8100.821429,0.214286,0.470107,04/07/2024


In [157]:
forecast_duration = 7 

In [158]:
import numpy as np

future_dates = pd.date_range(start=pd.Timestamp.today(), periods=forecast_duration, freq='D')
future_dates = future_dates.strftime('%m/%d/%Y')
future_df = pd.DataFrame({'date': future_dates})
for col in df.columns:
    if col != 'date':
        future_df[col] = np.nan
df_combined = pd.concat([df, future_df], ignore_index=True)

# Step 6: Forward fill the NaN values with the last known value
df_combined = df_combined.ffill()
df_combined

,time_index,topic_15_lag_1d,High_Low_Ratio_RollingAvg_14d,vader_sentiment_RollingAvg_3d,Vol._RollingAvg_14d,NORP_russian_RollingAvg_5d,Low_RollingAvg_14d,Vol._RollingAvg_3d,Vol._lag_10d,Low,...,High_Low_Ratio_RollingAvg_3d,textblob_sentiment_RollingAvg_7d,Low_lag_1d,Price_Range,Low_RollingAvg_3d,LAW_west ham_RollingAvg_5d,Low_RollingAvg_7d,ORDINAL_first_RollingAvg_14d,topic_15_RollingAvg_14d,date
0,2012-01-28,0.045760,1.011591,-0.044376,793.514555,0.143903,6829.312064,793.514555,793.514555,5728.96,...,1.011591,0.065850,6829.312064,66.24,6829.312064,0.004347,6829.312064,0.452986,0.045760,28/01/2012
1,2012-01-29,0.129553,1.011591,-0.044376,793.514555,0.143903,6829.312064,793.514555,793.514555,5728.96,...,1.011591,0.065850,5728.960000,66.24,6829.312064,0.004347,6829.312064,0.452986,0.045760,29/01/2012
2,2012-01-30,0.029583,1.011591,0.990100,793.514555,0.143903,6829.312064,786.066667,793.514555,5651.56,...,1.012538,0.065850,5728.960000,81.89,5703.160000,0.004347,6829.312064,0.452986,0.045760,30/01/2012
3,2012-01-31,0.108482,1.011591,0.984100,793.514555,0.143903,6829.312064,862.716667,793.514555,5671.09,...,1.012165,0.065850,5651.560000,59.23,5683.870000,0.004347,6829.312064,0.452986,0.045760,31/01/2012
4,2012-02-01,0.058440,1.011591,0.984967,793.514555,0.200000,6829.312064,927.783333,793.514555,5680.67,...,1.014769,0.065850,5671.090000,110.05,5667.773333,0.000000,6829.312064,0.452986,0.045760,01/02/2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4373,2024-08-04,0.538946,1.010936,-0.764233,942.827143,0.200000,7997.923571,962.880000,898.820000,8158.03,...,1.011235,-0.005186,8170.970000,125.33,8176.420000,0.000000,8120.301429,0.142857,0.430569,09/04/2024
4374,2024-08-04,0.538946,1.010936,-0.764233,942.827143,0.200000,7997.923571,962.880000,898.820000,8158.03,...,1.011235,-0.005186,8170.970000,125.33,8176.420000,0.000000,8120.301429,0.142857,0.430569,09/05/2024
4375,2024-08-04,0.538946,1.010936,-0.764233,942.827143,0.200000,7997.923571,962.880000,898.820000,8158.03,...,1.011235,-0.005186,8170.970000,125.33,8176.420000,0.000000,8120.301429,0.142857,0.430569,09/06/2024
4376,2024-08-04,0.538946,1.010936,-0.764233,942.827143,0.200000,7997.923571,962.880000,898.820000,8158.03,...,1.011235,-0.005186,8170.970000,125.33,8176.420000,0.000000,8120.301429,0.142857,0.430569,09/07/2024


In [159]:
for col in df_clean.columns:
    if col in df_combined.columns:
        # Update df_combined where dates match
        df_combined.loc[df_clean.index, col] = df_clean[col]
df_combined.loc[df_clean.index, col] = price
df_combined[-forecast_duration:]

,time_index,topic_15_lag_1d,High_Low_Ratio_RollingAvg_14d,vader_sentiment_RollingAvg_3d,Vol._RollingAvg_14d,NORP_russian_RollingAvg_5d,Low_RollingAvg_14d,Vol._RollingAvg_3d,Vol._lag_10d,Low,...,textblob_sentiment_RollingAvg_7d,Low_lag_1d,Price_Range,Low_RollingAvg_3d,LAW_west ham_RollingAvg_5d,Low_RollingAvg_7d,ORDINAL_first_RollingAvg_14d,topic_15_RollingAvg_14d,date,PERSON_kelly
4371,2024-08-04,0.538946,1.010936,-0.764233,942.827143,0.2,7997.923571,962.88,898.82,8158.03,...,-0.005186,8170.97,125.33,8176.42,0.0,8120.301429,0.142857,0.430569,09/02/2024,NaN
4372,2024-08-04,0.538946,1.010936,-0.764233,942.827143,0.2,7997.923571,962.88,898.82,8158.03,...,-0.005186,8170.97,125.33,8176.42,0.0,8120.301429,0.142857,0.430569,09/03/2024,NaN
4373,2024-08-04,0.538946,1.010936,-0.764233,942.827143,0.2,7997.923571,962.88,898.82,8158.03,...,-0.005186,8170.97,125.33,8176.42,0.0,8120.301429,0.142857,0.430569,09/04/2024,NaN
4374,2024-08-04,0.538946,1.010936,-0.764233,942.827143,0.2,7997.923571,962.88,898.82,8158.03,...,-0.005186,8170.97,125.33,8176.42,0.0,8120.301429,0.142857,0.430569,09/05/2024,NaN
4375,2024-08-04,0.538946,1.010936,-0.764233,942.827143,0.2,7997.923571,962.88,898.82,8158.03,...,-0.005186,8170.97,125.33,8176.42,0.0,8120.301429,0.142857,0.430569,09/06/2024,NaN
4376,2024-08-04,0.538946,1.010936,-0.764233,942.827143,0.2,7997.923571,962.88,898.82,8158.03,...,-0.005186,8170.97,125.33,8176.42,0.0,8120.301429,0.142857,0.430569,09/07/2024,NaN
4377,2024-08-04,0.538946,1.010936,-0.764233,942.827143,0.2,7997.923571,962.88,898.82,8158.03,...,-0.005186,8170.97,125.33,8176.42,0.0,8120.301429,0.142857,0.430569,09/08/2024,NaN
